# Tutorial: creación de ETLs con PySpark

In [ ]:
import os 
from pyspark.sql import functions as f, SparkSession, types as t
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql.functions import udf, col, length, isnan, when, count, regexp_replace
from datetime import datetime

In [ ]:
# Configuración servidor base de datos transaccional
# Recuerde usar Estudiante_i como usuario y la contraseña asigana en el excel de conexión a maquina virtual como contraseña
db_user = 'Estudiante_13_202314'
db_psswd = 'aabb1122'
source_db_connection_string = 'jdbc:mysql://157.253.236.116:8080/ProyectoTransaccional'
dest_db_connection_string = 'jdbc:mysql://157.253.236.116:8080/Proyecto_G3_202314'
# Driver de conexion
path_jar_driver = 'C:\Program Files (x86)\MySQL\Connector J 8.0\mysql-connector-java-8.0.28.jar'

In [ ]:
#Configuración de la sesión
conf=SparkConf() \
    .set('spark.driver.extraClassPath', path_jar_driver)

spark_context = SparkContext(conf=conf)
sql_context = SQLContext(spark_context)
spark = sql_context.sparkSession

In [4]:
def obterner_dataframe_desde_csv(_PATH, _sep):
    return spark.read.load(_PATH, format="csv", sep=_sep, inferSchema="true", header='true')

def obtener_dataframe_de_bd(db_connection_string, sql, db_user, db_psswd):
    df_bd = spark.read.format('jdbc')\
        .option('url', db_connection_string) \
        .option('dbtable', sql) \
        .option('user', db_user) \
        .option('password', db_psswd) \
        .option('driver', 'com.mysql.cj.jdbc.Driver') \
        .load()
    return df_bd

def guardar_db(db_connection_string, df, tabla, db_user, db_psswd):
    df.select('*').write.format('jdbc') \
      .mode('append') \
      .option('url', db_connection_string) \
      .option('dbtable', tabla) \
      .option('user', db_user) \
      .option('password', db_psswd) \
      .option('driver', 'com.mysql.cj.jdbc.Driver') \
      .save()

In [ ]:
Entidad 1

In [18]:
sql_MaxLongitudPista =  '''(SELECT MAX(longitud_pista) as maxLongitud FROM ProyectoTransaccional.aeropuertos) as maxLongitud'''
MaxLongitudPista = obtener_dataframe_de_bd(source_db_connection_string, sql_MaxLongitudPista, db_user, db_psswd)
MaxLongitudPista.show()

sql_MaxAnchoPista =  '''(SELECT MAX(ancho_pista) as maxAncho FROM ProyectoTransaccional.aeropuertos) as maxAncho'''
MaxAnchoPista = obtener_dataframe_de_bd(source_db_connection_string, sql_MaxAnchoPista, db_user, db_psswd)
MaxAnchoPista.show()

sql_MaxVuelosOrigen =  '''(SELECT MAX(numero_vuelos_origen) as maxVuelosOrigen FROM ProyectoTransaccional.aeropuertos) as maxVuelosOrigen'''
MaxVuelosOrigen = obtener_dataframe_de_bd(source_db_connection_string, sql_MaxVuelosOrigen, db_user, db_psswd)
MaxVuelosOrigen.show()

+-----------+
|maxLongitud|
+-----------+
|       5832|
+-----------+

+--------+
|maxAncho|
+--------+
|     136|
+--------+

+---------------+
|maxVuelosOrigen|
+---------------+
|        1495098|
+---------------+



In [63]:
sql_MiniDimensionAeropuerto = '''(SELECT ROW_NUMBER() OVER (ORDER BY sigla) AS idMini_DWH, 
        case 
            when ((SELECT MAX(longitud_pista) FROM ProyectoTransaccional.aeropuertos) / longitud_pista) < 25 then 'R1'
            when (((SELECT MAX(longitud_pista) FROM ProyectoTransaccional.aeropuertos) / longitud_pista) >= 25 and  ((SELECT MAX(longitud_pista) FROM ProyectoTransaccional.aeropuertos) / longitud_pista) < 50) then 'R2'
            when (((SELECT MAX(longitud_pista) FROM ProyectoTransaccional.aeropuertos) / longitud_pista) >= 50 and  ((SELECT MAX(longitud_pista) FROM ProyectoTransaccional.aeropuertos) / longitud_pista) < 75) then 'R3'
            else 'R4'
        end as rangoLongitudPista,
        case 
            when ((SELECT MAX(ancho_pista) FROM ProyectoTransaccional.aeropuertos) / ancho_pista) < 25 then 'R1'
            when (((SELECT MAX(ancho_pista) FROM ProyectoTransaccional.aeropuertos) / ancho_pista) >= 25 and  ((SELECT MAX(ancho_pista) FROM ProyectoTransaccional.aeropuertos) / ancho_pista) < 50) then 'R2'
            when (((SELECT MAX(ancho_pista) FROM ProyectoTransaccional.aeropuertos) / ancho_pista) >= 50 and  ((SELECT MAX(ancho_pista) FROM ProyectoTransaccional.aeropuertos) / ancho_pista) < 75) then 'R3'
            else 'R4'
        end as rangoAnchoPista,
        
        case 
            when ((SELECT MAX(numero_vuelos_origen) FROM ProyectoTransaccional.aeropuertos) / numero_vuelos_origen) < 25 then 'R1'
            when (((SELECT MAX(numero_vuelos_origen) FROM ProyectoTransaccional.aeropuertos) / numero_vuelos_origen) >= 25 and  ((SELECT MAX(numero_vuelos_origen) FROM ProyectoTransaccional.aeropuertos) / numero_vuelos_origen) < 50) then 'R2'
            when (((SELECT MAX(numero_vuelos_origen) FROM ProyectoTransaccional.aeropuertos) / numero_vuelos_origen) >= 50 and  ((SELECT MAX(numero_vuelos_origen) FROM ProyectoTransaccional.aeropuertos) / numero_vuelos_origen) < 75) then 'R3'
            else 'R4'
        end as rangoNumeroVuelosOrigen,
        clase, tipo, sigla as siglaAeropuerto FROM ProyectoTransaccional.aeropuertos) as MiniDimensionAeropuerto'''


MiniDimensionAeropuerto = obtener_dataframe_de_bd(source_db_connection_string, sql_MiniDimensionAeropuerto, db_user, db_psswd)
MiniDimensionAeropuerto.show()

+----------+------------------+---------------+-----------------------+-----+----------+---------------+
|idMini_DWH|rangoLongitudPista|rangoAnchoPista|rangoNumeroVuelosOrigen|clase|      tipo|siglaAeropuerto|
+----------+------------------+---------------+-----------------------+-----+----------+---------------+
|         1|                R1|             R1|                     R1|   1A|   Privado|            7FO|
|         2|                R1|             R1|                     R1|   1A|   Privado|            7FO|
|         3|                R1|             R1|                     R1|   1A|   Privado|            7FO|
|         4|                R1|             R1|                     R1|   1A|Fumigación|            7FU|
|         5|                R2|             R1|                     R1|   1A|Fumigación|            7FU|
|         6|                R1|             R1|                     R1|   1A|Fumigación|            7FU|
|         7|                R1|             R1|        

In [29]:
Trandormaciones 


NameError: name 'Trandormaciones' is not defined

In [64]:
MiniDimensionAeropuerto_0 = [(0,'NA', 'NA', 'NA', 'NA', 'NA', 'NA')]
columns = ['idMini_DWH','rangoLongitudPista','rangoAnchoPista','rangoNumeroVuelosOrigen','clase','tipo', 'siglaAeropuerto']
MiniDimensionAeropuerto_0 = spark.createDataFrame(data = MiniDimensionAeropuerto_0, schema = columns)
MiniDimensionAeropuerto_0.show()

+----------+------------------+---------------+-----------------------+-----+----+---------------+
|idMini_DWH|rangoLongitudPista|rangoAnchoPista|rangoNumeroVuelosOrigen|clase|tipo|siglaAeropuerto|
+----------+------------------+---------------+-----------------------+-----+----+---------------+
|         0|                NA|             NA|                     NA|   NA|  NA|             NA|
+----------+------------------+---------------+-----------------------+-----+----+---------------+



In [66]:
MiniDimensionAeropuerto = MiniDimensionAeropuerto.union(MiniDimensionAeropuerto_0)
MiniDimensionAeropuerto.show().orderBy(col('idMini_DWH'))

+----------+------------------+---------------+-----------------------+-----+----------+---------------+
|idMini_DWH|rangoLongitudPista|rangoAnchoPista|rangoNumeroVuelosOrigen|clase|      tipo|siglaAeropuerto|
+----------+------------------+---------------+-----------------------+-----+----------+---------------+
|         1|                R1|             R1|                     R1|   1A|   Privado|            7FO|
|         2|                R1|             R1|                     R1|   1A|   Privado|            7FO|
|         3|                R1|             R1|                     R1|   1A|   Privado|            7FO|
|         4|                R1|             R1|                     R1|   1A|Fumigación|            7FU|
|         5|                R2|             R1|                     R1|   1A|Fumigación|            7FU|
|         6|                R1|             R1|                     R1|   1A|Fumigación|            7FU|
|         7|                R1|             R1|        

AttributeError: 'NoneType' object has no attribute 'orderBy'

In [39]:
MiniDimensionAeropuerto = MiniDimensionAeropuerto.withColumn('idMini_DWH',col('idMini_DWH').cast('int')).orderBy(col('idMini_DWH'))
MiniDimensionAeropuerto.show()

+----------+------------------+---------------+-----------------------+-----+----------+
|idMini_DWH|rangoLongitudPista|rangoAnchoPista|rangoNumeroVuelosOrigen|clase|      tipo|
+----------+------------------+---------------+-----------------------+-----+----------+
|         0|                NA|             NA|                     NA|   NA|        NA|
|         0|                NA|             NA|                     NA|   NA|        NA|
|         0|                NA|             NA|                     NA|   NA|        NA|
|         0|                NA|             NA|                     NA|   NA|        NA|
|         0|                NA|             NA|                     NA|   NA|        NA|
|         1|                R1|             R1|                     R1|   1A|   Privado|
|         2|                R1|             R1|                     R1|   1A|   Privado|
|         3|                R1|             R1|                     R1|   1A|   Privado|
|         4|         

In [40]:
# CARGUE
guardar_db(dest_db_connection_string,MiniDimensionAeropuerto,'Proyecto_G3_202314.MiniDimensionAeropuerto', db_user, db_psswd)

In [ ]:
Entidad 2

In [46]:
sql_TipoVuelo = '''(SELECT ROW_NUMBER() OVER (ORDER BY origen) AS idTipoVuelo_DWH,
		ROW_NUMBER() OVER (ORDER BY origen) AS idTipoVuelo_T,
          case 
            when (tipo_vuelo) = 'R' then 'Regular'
            when (tipo_vuelo) = 'T' then 'Taxi'
            when (tipo_vuelo) = 'C' then 'Caracter'
            else 'Adicionales'
        end as nombreTipo,
       tipo_equipo as tipoEquipo FROM ProyectoTransaccional.vuelos) as TipoVuelo'''


TipoVuelo = obtener_dataframe_de_bd(source_db_connection_string, sql_TipoVuelo, db_user, db_psswd)
TipoVuelo.show()

+---------------+-------------+----------+----------+
|idTipoVuelo_DWH|idTipoVuelo_T|nombreTipo|tipoEquipo|
+---------------+-------------+----------+----------+
|              1|            1|      Taxi|      PA34|
|              2|            2|      Taxi|      C210|
|              3|            3|      Taxi|      B190|
|              4|            4|      Taxi|      C402|
|              5|            5|      Taxi|      C402|
|              6|            6|      Taxi|      C402|
|              7|            7|      Taxi|      C402|
|              8|            8|      Taxi|      C402|
|              9|            9|      Taxi|      PA34|
|             10|           10|      Taxi|      C402|
|             11|           11|      Taxi|      PA34|
|             12|           12|      Taxi|      C402|
|             13|           13|      Taxi|      PA34|
|             14|           14|      Taxi|      PA31|
|             15|           15|      Taxi|      PA31|
|             16|           

In [ ]:
Trasformaciones

In [47]:
TipoVuelo_0 = [(0, 0, 'NA', 'NA')]
columns = ['idTipoVuelo_DWH','idTipoVuelo_T','nombreTipo','tipoEquipo']
TipoVuelo_0 = spark.createDataFrame(data = TipoVuelo_0, schema = columns)
TipoVuelo_0.show()

+---------------+-------------+----------+----------+
|idTipoVuelo_DWH|idTipoVuelo_T|nombreTipo|tipoEquipo|
+---------------+-------------+----------+----------+
|              0|            0|        NA|        NA|
+---------------+-------------+----------+----------+



In [48]:
TipoVuelo = TipoVuelo.union(TipoVuelo_0)
TipoVuelo.show()

+---------------+-------------+----------+----------+
|idTipoVuelo_DWH|idTipoVuelo_T|nombreTipo|tipoEquipo|
+---------------+-------------+----------+----------+
|              1|            1|      Taxi|      PA34|
|              2|            2|      Taxi|      C210|
|              3|            3|      Taxi|      B190|
|              4|            4|      Taxi|      C402|
|              5|            5|      Taxi|      C402|
|              6|            6|      Taxi|      C402|
|              7|            7|      Taxi|      C402|
|              8|            8|      Taxi|      C402|
|              9|            9|      Taxi|      PA34|
|             10|           10|      Taxi|      C402|
|             11|           11|      Taxi|      PA34|
|             12|           12|      Taxi|      C402|
|             13|           13|      Taxi|      PA34|
|             14|           14|      Taxi|      PA31|
|             15|           15|      Taxi|      PA31|
|             16|           

AttributeError: 'NoneType' object has no attribute 'orderBy'

In [49]:
TipoVuelo = TipoVuelo.withColumn('idTipoVuelo_DWH',col('idTipoVuelo_DWH').cast('int')).orderBy(col('idTipoVuelo_DWH'))
TipoVuelo.show()

+---------------+-------------+----------+----------+
|idTipoVuelo_DWH|idTipoVuelo_T|nombreTipo|tipoEquipo|
+---------------+-------------+----------+----------+
|              0|            0|        NA|        NA|
|              1|            1|      Taxi|      PA34|
|              2|            2|      Taxi|      C210|
|              3|            3|      Taxi|      B190|
|              4|            4|      Taxi|      C402|
|              5|            5|      Taxi|      C402|
|              6|            6|      Taxi|      C402|
|              7|            7|      Taxi|      C402|
|              8|            8|      Taxi|      C402|
|              9|            9|      Taxi|      PA34|
|             10|           10|      Taxi|      C402|
|             11|           11|      Taxi|      PA34|
|             12|           12|      Taxi|      C402|
|             13|           13|      Taxi|      PA34|
|             14|           14|      Taxi|      PA31|
|             15|           

In [50]:
# CARGUE
guardar_db(dest_db_connection_string, TipoVuelo,'Proyecto_G3_202314. TipoVuelo', db_user, db_psswd)